In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import yaml
import glob
import rawpy
import zivid
import os
import open3d as o3d
from tqdm import trange

In [ ]:
path = "data"
zdf1 = "20221117_1013_calib_test.zdf"
cam2 = "DSCF9210.RAF" 

with open(os.path.join(path, "calib_result.yaml")) as f:
    calib = yaml.safe_load(f)

k1 = np.array(calib["k2"])
d1 = np.array(calib["d2"])
# k2 = np.array(calib["cameraMatrix2"])
# d2 = np.array(calib["distCoeffs2"])
R = np.array(calib["R"])
T = np.array(calib["T"])
E = np.array(calib["E"])
F = np.array(calib["F"])
RT = np.hstack((R, T))

In [ ]:
# load zivid point cloud
with zivid.Application() as app:
    frame = zivid.Frame(os.path.join(path, zdf1))
    point_cloud = frame.point_cloud()
    xyz = point_cloud.copy_data("xyz")
    rgba = point_cloud.copy_data("rgba")
    xyzrgba = point_cloud.copy_data("xyzrgba")
    rgb = rgba[..., :3]

print("xyz.shape:", xyz.shape)
print("rgb.shape:", rgb.shape)
print("xyzrgba.shape:", xyzrgba.shape)

In [ ]:
img_f = rawpy.imread(os.path.join(path, cam2)).postprocess(use_camera_wb=True, no_auto_bright=True, output_bps=8)[:8733, :11644]

# rotate xyz 180 degrees
xyz_ = xyz.copy()
xyz_[:, :, :2] = np.negative(xyz[:, :, :2])
# xyz = xyz[::-1, ::-1, :]
xyz_ = xyz_[:, 160:-160, :]
rgb_ = rgb[:, 160:-160, :]

# remove NaNs in xyz
xyz_ = xyz_.reshape(-1, 3)
rgb_= rgb_.reshape(-1, 3)

rgb_ = rgb_[~np.isnan(xyz_).any(axis=1)]
xyz_ = xyz_[~np.isnan(xyz_).any(axis=1)]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_)
pcd.colors = o3d.utility.Vector3dVector(rgb_ / 255)
o3d.visualization.draw_geometries([pcd])

In [ ]:
# project xyz to img_f with k1 and RT
uv_ = []

for i in trange(len(xyz_)):
    uv = cv2.projectPoints(xyz_[i], R, T, k1, d1)[0]
    uv_.append(uv)

uv_ = np.array(uv_).reshape(-1, 2)
print(uv_)


In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_)

# fill the color of the point cloud with img_f values at uv_
pcd.colors = o3d.utility.Vector3dVector(img_f[uv_[:, 1].astype(int), uv_[:, 0].astype(int), :]/255)
# save pcd to ply
o3d.io.write_point_cloud(os.path.join(path, "pcd.ply"), pcd)
# visualize pcd
o3d.visualization.draw_geometries([pcd])